In [ ]:
import numpy as np
import gwdistributions
from gwdistributions import distributions, event, generators, transforms
from gwdistributions.utils import Parameters
from gwdetectors import detector
from gwdetectors import cache

To Fix:
Errors:
 - event doesnt have strain attribute when trying to run observedSNR, not sure I'm applying the SNR condition right at all

Stuff I'm unsure of:
 - I was sampling in azimuth and zenith because bilby_pipe seemed to prefer that, Reed's code uses RA/Dec, not sure what the best way to remedy that is
 - Waveform default is IMRPhenomXPHM in Reed's code, not sure how to change it to IMRPhenomNSBH
    DEFAULT_DELTAT = 1/4096, I uss 0.2
    DEFAULT_FLOW = 10, I used 20
    DEFAULY_FHIGH = 2048, I used 1000
    DEFAULT_DELTAF = 0.125, don't know what this is
    DEFAULT_FREF = 10, I used 50


In [14]:
paramsm11 = {'min_mass1_source':2.5, 'max_mass1_source':17, 'mean_mass1_source':11.053, 'stdv_mass1_source':2.909}
paramsm12 = {'min_mass1_source':2.5, 'max_mass1_source':17, 'mean_mass1_source':13.687, 'stdv_mass1_source':0.570}
m11 = distributions.TruncatedGaussianMass1()
print(m11.param_names)
m11.parameters = paramsm11
m12 = distributions.TruncatedGaussianMass1()
m12.parameters = paramsm12
m1 = distributions.Mixture((m11, 0.895), (m12, 0.105))

m2 = distributions.TruncatedGaussianMass2()
paramsm2 = {'min_mass2_source':1.69, 'max_mass2_source':1.71, 'mean_mass2_source':1.7, 'stdv_mass2_source':1}
m2.parameters = paramsm2

# luminosity_distance = distributions.PowerLawLuminosityDistance()
# print(luminosity_distance.param_names)
# paramslum = {'min_luminosity_distance':20, 'max_luminosity_distance':1000, 'pow_luminosity_distance':2.7}
# luminosity_distance.parameters = paramslum

z = distributions.PowerLaw1plusRedshift()
paramsz = {'min_redshift':0, 'max_redshift':1, 'pow_redshift':1.7} # Considering the (1+z) in the denominator
z.parameters = paramsz

angle1 = distributions.AlignedSpin1PolarAngle()
chi_11 = distributions.TruncatedGaussianSpin1Magnitude()
print(chi_11.param_names)
paramschi11 = {'min_spin1_magnitude':-1, 'max_spin1_magnitude':1, 'mean_spin1_magnitude':-0.5, 'stdv_spin1_magnitude':0.08}
chi_11.parameters = paramschi11
chi_12 = distributions.TruncatedGaussianSpin1Magnitude()
paramschi12 = {'min_spin1_magnitude':-1, 'max_spin1_magnitude':1, 'mean_spin1_magnitude':0, 'stdv_spin1_magnitude':0.01}
chi_12.parameters = paramschi12
chi_13 = distributions.TruncatedGaussianSpin1Magnitude()
paramschi13 = {'min_spin1_magnitude':-1, 'max_spin1_magnitude':1, 'mean_spin1_magnitude':0.5, 'stdv_spin1_magnitude':0.08}
chi_13.parameters = paramschi13
chi_1 = distributions.Mixture((chi_12, 0.85), (chi_11, 0.075), (chi_13, 0.075))

chi_2 = distributions.ZeroSpin2Magnitude()

# lambda_1 = distributions.Zero()
# lambda_2 = distributions.Zero()
# theta_jn = distributions.Uniform() # Should be Sine distribution
# theta_jn.parameters = {'min_variate':0, 'max_variate':np.pi}
# psi = distributions.Uniform()
# psi.parameters = {'min_variate':0, 'max_variate':2*np.pi}
# phase = distributions.Uniform()
# phase.parameters = {'min_variate':0, 'max_variate':2*np.pi}
orientation = distributions.RandomOrientation()
RAdec = distributions.location.IsotropicRADec()
geocent_time = distributions.FixedTime()
geocent_time.parameters = {'geocenter_time':1240215503.0}


dists = [m1, m2, z, angle1, chi_1, chi_2, orientation, RAdec, geocent_time]

transform = [transforms.Redshift2LuminosityDistance()]

print(cache.detector.KNOWN_DETECTORS)
L = cache.detector.DETECTORS['L_aligo-design']
H = cache.detector.DETECTORS['H_aligo-design']
print(type(L))
network1 = detector.Network(L,H)
detect_prob = [transforms.OptimalSNR(
                                    network=network1,
                                    snr_flow = 10,
                                    )]

# detect_prob = [transforms.DetectionProbabilityGivenOptimalSNR(network=network1,
#                                                                 snr_flow = 10)]
print(detect_prob[0].network)
gen = generators.EventGenerator(distributions = dists, 
                                conditionals = detect_prob,
                                transforms = transform)
gen.flush_events
calls = gen.generate(n_event = 100)
print(gen.total_generated)
print(gen.events)
print(calls)
gen.write()

('min_mass1_source', 'max_mass1_source', 'mean_mass1_source', 'stdv_mass1_source')
('min_spin1_magnitude', 'max_spin1_magnitude', 'mean_spin1_magnitude', 'stdv_spin1_magnitude')
['CE@H_ce-design', 'CE@L_ce-design', 'H_aligo-design', 'H_aplus-design', 'L_aligo-design', 'L_aplus-design', 'V_advirgo-design']
<class 'gwdetectors.detector.twoarmdetector.TwoArmDetector'>
Network(
  TwoArmDetector( L_aligo-design, location = (-2.477582e-04, -1.833363e-02, 1.075496e-02) sec, arms = ( (-1.273647e-05, -1.889051e-06, -3.498275e-06) sec, (3.972636e-06, -6.509975e-06, -1.094817e-05) sec, ) psd = OneSidedPowerSpectralDensity(aligo-design, freq within [5.000000e+00 5.000000e+03])), long-wavelength approximation = False, ),
  TwoArmDetector( H_aligo-design, location = (-7.209704e-03, -1.279117e-02, 1.534512e-02) sec, arms = ( (-2.987302e-06, 1.067179e-05, 7.430539e-06) sec, (-1.219481e-05, 3.481614e-07, -5.402717e-06) sec, ) psd = OneSidedPowerSpectralDensity(aligo-design, freq within [5.000000e+00 5.

AssertionError: Event does not have attribute "strain_L_aligo-design"